In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('NIFTY_2010.csv')

In [7]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [8]:
df

,Instrument,Date,Time,Open,High,Low,Close
0,NIFTY,20100104,09:31,5201.00,5224.00,5167.00,5216.00
1,NIFTY,20100104,09:32,5216.00,5216.00,5215.00,5215.00
2,NIFTY,20100104,09:33,5215.00,5216.00,5215.00,5216.00
3,NIFTY,20100104,09:34,5216.00,5217.00,5216.00,5217.00
4,NIFTY,20100104,09:35,5217.00,5217.00,5216.00,5217.00
...,...,...,...,...,...,...,...
96678,NIFTY,20101231,15:26,6133.55,6133.60,6130.85,6130.85
96679,NIFTY,20101231,15:27,6130.65,6131.05,6126.35,6127.50
96680,NIFTY,20101231,15:28,6127.75,6129.50,6127.25,6128.15
96681,NIFTY,20101231,15:29,6128.00,6129.50,6127.70,6129.20


In [9]:

datecount = dict()
for i in range(96683):
    date = df['Date'][i] - 20100104
    if(datecount.get(date) is not None):
        datecount[date]+= 1
    else:
        datecount[date] = 1
    
print(datecount)


{0: 360, 1: 390, 2: 390, 3: 390, 4: 390, 7: 390, 8: 390, 9: 390, 10: 390, 11: 390, 14: 390, 15: 390, 16: 390, 17: 390, 18: 390, 21: 390, 23: 382, 24: 390, 25: 390, 97: 390, 98: 390, 99: 390, 100: 390, 101: 378, 102: 90, 104: 390, 105: 389, 106: 390, 107: 390, 111: 390, 112: 390, 113: 390, 114: 390, 115: 390, 118: 390, 119: 390, 120: 390, 121: 390, 122: 390, 198: 390, 199: 390, 200: 390, 201: 390, 204: 390, 205: 390, 206: 390, 207: 390, 208: 390, 211: 390, 212: 390, 213: 390, 214: 390, 215: 390, 218: 390, 219: 390, 221: 390, 222: 390, 225: 390, 226: 390, 227: 390, 297: 390, 301: 390, 302: 390, 303: 390, 304: 390, 305: 390, 308: 390, 309: 390, 311: 390, 312: 390, 315: 390, 316: 390, 317: 390, 318: 390, 319: 390, 322: 390, 323: 390, 324: 390, 325: 390, 326: 390, 399: 390, 400: 390, 401: 390, 402: 390, 403: 390, 406: 390, 407: 390, 408: 390, 409: 390, 410: 390, 413: 390, 414: 390, 415: 390, 416: 390, 417: 390, 420: 390, 421: 390, 422: 390, 423: 390, 424: 390, 427: 390, 497: 390, 498: 390, 

In [55]:
df.Date.nunique()

251

In [10]:
def convert_to_day(df):
    NoOfDays = df['Date'].nunique()
    newdf = pd.DataFrame({'Date' : [],'Open' : [],'High' : [],'Low': [], 'Close' : []})
    i = 0
    while i < 96683:
        # newdf.columns = ['Date','Open','High','Low','Close']
        day   = df['Date'][i]
        count = df['Date'].value_counts()[day]
        open  = df['Open'][i]  
        close = df['Close'][count +i - 1]
        high = 0
        low = 100000
        for j in range(count):
            if(low > df['Low'][i+ j]):
                low = df['Low'][i + j]
            if(high < df['High'][i+j]):
                high= df['High'][i+j]
        i = i + count
        new_record = pd.DataFrame([{'Date' : day,'Open' : open, 'High' : high, 'Low' : low, 'Close' : close}])
        newdf = pd.concat([newdf,new_record],ignore_index=True)
    return newdf
daily = convert_to_day(df)
print(daily)

           Date     Open     High      Low    Close
0    20100104.0  5201.00  5238.00  5167.00  5235.00
1    20100105.0  5277.00  5288.00  5242.00  5274.00
2    20100106.0  5278.00  5311.00  5260.00  5281.00
3    20100107.0  5281.80  5302.55  5244.75  5256.60
4    20100108.0  5264.25  5276.75  5234.70  5247.00
..          ...      ...      ...      ...      ...
246  20101227.0  6013.30  6045.75  5991.10  5993.00
247  20101228.0  6005.35  6010.90  5982.25  5995.90
248  20101229.0  6005.15  6067.55  6002.85  6063.05
249  20101230.0  6062.35  6106.40  6062.35  6090.50
250  20101231.0  6105.10  6147.30  6103.55  6127.70

[251 rows x 5 columns]


In [23]:
def convert_to_15_min(df):
    NoOzfDays = df['Date'].nunique()
    newdf = pd.DataFrame({'Date' : [],'Start 15': [],'Open' : [],'High' : [],'Low': [], 'Close' : []})
    i = 0
    size = int(df.shape[0])
    while i < size:
        # newdf.columns = ['Date','Open','High','Low','Close']
        count = 0
        starttime = df['Time'][i]
        day   = df['Date'][i]
        if(i >= (size - 15)):
            count = size - i
        elif df['Date'][i] == df['Date'][i+15]:
            count = 15
        else:
            j = i
            while df['Date'][i] == df['Date'][j] and count < 15:
                count+=1
                j+=1
        open  = df['Open'][i]  
        close = df['Close'][count +i - 1]
        high = 0
        low = 100000
        for j in range(count):
            if(i+j >=size):
                break
            if(low > df['Low'][i+ j]):
                low = df['Low'][i + j]
                high= df['High'][i+j]
        i = i + count
        new_record = pd.DataFrame([{'Date' : day,'Start 15': starttime,'Open' : open, 'High' : high, 'Low' : low, 'Close' : close}])
        newdf = pd.concat([newdf,new_record],ignore_index=True)
    return newdf
minutes_15 = convert_to_15_min(df)
print(minutes_15)

            Date Start 15     Open     High      Low   Close
0     20100104.0    09:31  5201.00  5224.00  5167.00  5215.0
1     20100104.0    09:46  5215.00  5215.00  5213.00  5220.0
2     20100104.0    10:01  5220.00  5219.00  5218.00  5219.0
3     20100104.0    10:16  5220.00  5218.00  5216.00  5222.0
4     20100104.0    10:31  5222.00  5222.00  5221.00  5226.0
...          ...      ...      ...      ...      ...     ...
6487  20101231.0    14:23  6138.30  6135.20  6133.35  6140.7
6488  20101231.0    14:38  6140.55  6138.35  6135.95  6136.3
6489  20101231.0    14:53  6136.15  6131.65  6128.20  6139.2
6490  20101231.0    15:08  6139.45  6136.30  6133.70  6135.7
6491  20101231.0    15:23  6135.55  6131.50  6126.20  6127.7

[6492 rows x 6 columns]


In [24]:
def convert_to_5_min(df):
    NoOzfDays = df['Date'].nunique()
    newdf = pd.DataFrame({'Date' : [],'Start 5': [],'Open' : [],'High' : [],'Low': [], 'Close' : []})
    i = 0
    size = int(df.shape[0])
    while i < size:
        # newdf.columns = ['Date','Open','High','Low','Close']
        count = 0
        starttime = df['Time'][i]
        day   = df['Date'][i]
        if(i >= (size - 5)):
            count = size - i
        elif df['Date'][i] == df['Date'][i+5]:
            count = 5
        else:
            j = i
            while df['Date'][i] == df['Date'][j] and count < 5:
                count+=1
                j+=1
        open  = df['Open'][i]  
        close = df['Close'][count +i - 1]
        high = 0
        low = 100000
        for j in range(count):
            if(i+j >=size):
                break
            if(low > df['Low'][i+ j]):
                low = df['Low'][i + j]
                high= df['High'][i+j]
        i = i + count
        new_record = pd.DataFrame([{'Date' : day,'Start 5': starttime,'Open' : open, 'High' : high, 'Low' : low, 'Close' : close}])
        newdf = pd.concat([newdf,new_record],ignore_index=True)
    return newdf
minutes_5 = convert_to_5_min(df)
print(minutes_5)

             Date Start 5     Open     High      Low    Close
0      20100104.0   09:31  5201.00  5224.00  5167.00  5217.00
1      20100104.0   09:36  5218.00  5218.00  5214.00  5215.00
2      20100104.0   09:41  5215.00  5214.00  5212.00  5215.00
3      20100104.0   09:46  5215.00  5215.00  5213.00  5214.00
4      20100104.0   09:51  5214.00  5215.00  5213.00  5219.00
...           ...     ...      ...      ...      ...      ...
19373  20101231.0   15:08  6139.45  6136.25  6133.95  6134.80
19374  20101231.0   15:13  6134.70  6136.35  6134.20  6135.85
19375  20101231.0   15:18  6136.15  6136.30  6133.70  6135.70
19376  20101231.0   15:23  6135.55  6131.05  6126.35  6127.50
19377  20101231.0   15:28  6127.75  6131.50  6126.20  6127.70

[19378 rows x 6 columns]
